# Watcha Pedia 박스 오피스 영화 정보 수집하기

## 필요한 라이브러리 설치

In [14]:
!pip install lxml
!pip install requests
!pip install bs4
!pip install openpyxl
!pip install selenium

## 목표 URL 수집하기

In [5]:
import requests # HTTP request를 쉽게 구현해주는 라이브러리
watcha_url = "https://pedia.watcha.com/ko-KR/?domain=movie"# 크롤링 원하는 주소
response = requests.get(watcha_url)
response # 내가 원하는 것? https://docs.python-requests.org/en/latest/api/#requests.Response

<Response [200]>

In [7]:
from lxml import html # XML 또는 HTML을 Parsing해 주는 라이브러리
bo_urls = ["https://pedia.watcha.com"+i for i in html.fromstring(response.text).xpath('//*[@id="root"]/div/div[1]/section/div/section/div[1]/div[2]/div/div[1]/div/div/ul/li//a/@href')]

['https://pedia.watcha.com/ko-KR/contents/mO1jmAV',
 'https://pedia.watcha.com/ko-KR/contents/mdj20mq',
 'https://pedia.watcha.com/ko-KR/contents/m5Zl46z',
 'https://pedia.watcha.com/ko-KR/contents/mOgjGVz',
 'https://pedia.watcha.com/ko-KR/contents/md6aNEZ',
 'https://pedia.watcha.com/ko-KR/contents/m5Go91n',
 'https://pedia.watcha.com/ko-KR/contents/mW42rQw',
 'https://pedia.watcha.com/ko-KR/contents/mW42AGq',
 'https://pedia.watcha.com/ko-KR/contents/m5x1J2x',
 'https://pedia.watcha.com/ko-KR/contents/mOAkDQP']

### [Xpath 더 알아보기](https://www.w3schools.com/xml/xpath_syntax.asp)

## 각 URL에서 원하는 정보 획득하기

In [8]:
bo_list = []
for bo_url in bo_urls:
    movie_page = requests.get(bo_url)
    movie_title = [i.text for i in html.fromstring(movie_page.text).xpath('//*[@id="root"]/div/div[1]/section/div/div[2]/div/section/div[2]/div/div/div/div/h1')][0]
    movie_detail = [i for i in html.fromstring(movie_page.text).xpath('//*[@id="root"]/div/div[1]/section/div/div[2]/div/section/div[2]/div/div/div/div/div[1]/text()')][0]
    movie_desc  = [i for i in html.fromstring(movie_page.text).xpath('//*[@id="root"]/div/div[1]/section/div/div[2]/div/div/div/div[1]/div[1]/div/div/section[2]/div[2]/div/article/div[2]/div/text()')][0]
    movie_rate  = [i for i in html.fromstring(movie_page.text).xpath('//*[@id="root"]/div/div[1]/section/div/div[2]/div/section/div[2]/div/div/div/div/div[2]/text()')][0]
    movie_poster = [i.get('src') for i in html.fromstring(movie_page.text).xpath('//*[@id="root"]/div/div[1]/section/div/div[2]/div/section/div[1]/div[2]/div/div/div/div[1]/img')][0]
    movie_dict = {"movie_title":movie_title, "movie_detail":movie_detail,
                  "movie_desc":movie_desc, "movie_rate":movie_rate, "movie_poster":movie_poster}
    bo_list.append(movie_dict)


In [9]:
bo_list

[{'movie_title': '문폴',
  'movie_detail': '2022 ・ 판타지/액션/드라마 ・ 캐나다/미국/영국',
  'movie_desc': '1969년 인류의 위대한 도전, 달 착륙 성공 2022년 인류 최후의 재난, 달이 지구와 충돌한다! 궤도를 이탈한 달이 지구를 향해 떨어지기 시작하면서 지구의 중력과 모든 물리적인 법칙이 붕괴된다. 거대한 해일과 지진, 화산폭발, 쓰나미와 이상기후까지 상상조차 불가능했던 모든 재난으로 전 세계는 공포와 혼란에 빠진다. 달과 충돌까지 남은 시간은 단 30일. NASA 연구원 `파울러`(할리 베리), 전직 우주 비행사 `브라이언`(패트릭 윌슨), 그리고 우주 덕후 `KC`(존 브래들리)는 달을 막을 방법을 찾기 위해 마지막 우주선에 오른다. 인류 멸망 D-30일, 추락하는 달을 반드시 멈춰야 한다',
  'movie_rate': '평균 ★2.2 (698명)',
  'movie_poster': 'https://an2-img.amz.wtchn.net/image/v2/E8ZdVl5FnCa3PbIiLgjkew.jpg?jwt=ZXlKaGJHY2lPaUpJVXpJMU5pSjkuZXlKdmNIUnpJanBiSW1SZk1qZ3dlRFF3TUhFNE1DSmRMQ0p3SWpvaUwzWXlMM04wYjNKbEwybHRZV2RsTHpFMk5EWXhORE0zTlRneU1USXpOalExTlRZaWZRLjEzM3lRcGNPZVRnRDJoMDdOWW5pNk0zSDdDZk5TN1dkcmVGemNBMDZQejQ'},
 {'movie_title': '뜨거운 피',
  'movie_detail': '2020 ・ 범죄/드라마 ・ 한국',
  'movie_desc': '부산 변두리 작은 포구 `구암`의 절대적인 주인 `손영감`(김갑수),\xa0그의 밑에서 수년간 수족으로 일해온 `희수`(정우)는\xa0무엇 하나 이뤄낸 것 없이, 큰돈 한번 만져보지 못한 채 반복되는 건달 짓이 지긋지긋하다.\xa0 1993년, 범죄와의 전쟁 이후 새로운 구역

In [22]:
# 제대로 수집 됐는지 확인하기
from IPython.display import Image, display # Jupyter notebook에서 이미지를 보여주는 라이브러리
Image(url=bo_list[5]['movie_poster'])

## BeautifulSoup4으로 한다면?

In [23]:
from bs4 import BeautifulSoup # https://www.crummy.com/software/BeautifulSoup/bs4/doc/
for bo_url in bo_urls:
    movie_page = requests.get(bo_url).text
    soup = BeautifulSoup(movie_page, 'html.parser')
    movie_title = soup.select_one('#root > div > div.css-1xm32e0 > section > div > div.css-1ihluk0-Content.e1ezac431 > div > section > div.css-1p7n6er-Pane.e1svyhwg15 > div > div > div > div > h1')
    movie_title = movie_title.get_text()
    print(movie_title)

문폴
뜨거운 피
스펜서
이상한 나라의 수학자
더 배트맨
극장판 주술회전 0
메리 미
도어맨
하늘의 푸르름을 아는 사람이여
고양이들의 아파트


## pandas DataFrame으로 만들기

In [24]:
import pandas as pd

In [25]:
box_office = pd.DataFrame(bo_list)
box_office

,movie_title,movie_detail,movie_desc,movie_rate,movie_poster
0,문폴,2022 ・ 판타지/액션/드라마 ・ 캐나다/미국/영국,"1969년 인류의 위대한 도전, 달 착륙 성공 2022년 인류 최후의 재난, 달이 ...",평균 ★2.2 (567명),https://an2-img.amz.wtchn.net/image/v2/E8ZdVl5...
1,뜨거운 피,2020 ・ 범죄/드라마 ・ 한국,"부산 변두리 작은 포구 `구암`의 절대적인 주인 `손영감`(김갑수), 그의 밑에서 ...",,https://an2-img.amz.wtchn.net/image/v2/alJx8BY...
2,스펜서,2021 ・ 전기/드라마 ・ 영국/독일/미국,"1991년, 왕실 가족이 샌드링엄 별장에 모여서 보내는 크리스마스. 연휴 3일간 이...","평균 ★3.6 (1,456명)",https://an2-img.amz.wtchn.net/image/v2/_8BzRXr...
3,이상한 나라의 수학자,2020 ・ 드라마 ・ 한국,“정답보다 중요한 건 답을 찾는 과정이야” 학문의 자유를 갈망하며 탈북한 천재 수학...,"평균 ★2.9 (1,712명)",https://an2-img.amz.wtchn.net/image/v2/hsg-TCp...
4,더 배트맨,2022 ・ 액션/범죄/드라마 ・ 미국,영웅이 될 것인가 악당이 될 것인가 운명을 결정할 선택만이 남았다 지난 2년간 고담...,평균 ★3.5 (2만명),https://an2-img.amz.wtchn.net/image/v2/NdPk_f6...
5,극장판 주술회전 0,2021 ・ 애니메이션/액션/모험 ・ 일본,어릴 적 소꿉친구인 오리모토 리카를 교통사고로 눈앞에서 잃은 옷코츠 유타. “약속해...,"평균 ★3.8 (3,946명)",https://an2-img.amz.wtchn.net/image/v2/5CSy5j_...
6,메리 미,2022 ・ 로맨스/코미디 ・ 미국,선결혼 후연애! 슈퍼스타 X 슈퍼노멀 커플 탄생! 슈퍼스타 `캣 발데즈’는 화려한 ...,평균 ★2.8 (65명),https://an2-img.amz.wtchn.net/image/v2/nPAFcWi...
7,도어맨,2020 ・ 액션/스릴러 ・ 미국,해병대 복무 중 테러집단의 공격으로 대사와 가족을 잃은 알리(루비 로즈)는 제대 후...,평균 ★2.2 (165명),https://an2-img.amz.wtchn.net/image/v2/-a_wFi_...
8,하늘의 푸르름을 아는 사람이여,2019 ・ 판타지/로맨스/애니메이션 ・ 일본,"“후회하지 않기 위해, 다시 널 만나러 왔어” ​ 산으로 둘러 싸인 시골 마을, 그...",평균 ★3.2 (365명),https://an2-img.amz.wtchn.net/image/v2/mPP0dI5...
9,고양이들의 아파트,2020 ・ 다큐멘터리 ・ 한국,"서울 동쪽 끝, 거대한 아파트 단지. 그곳은 오래도록 고양이들과 사람들이 함께 마음...",평균 ★3.3 (115명),https://an2-img.amz.wtchn.net/image/v2/c3TXrIj...


## movie_detail에서 년도, 장르, 국적 분리하기 

In [26]:
box_office['movie_year'] = box_office['movie_detail'].apply(lambda x: x.split(" ・ ")[0])
box_office['movie_genre'] = box_office['movie_detail'].apply(lambda x: x.split(" ・ ")[1])
box_office['movie_nation'] = box_office['movie_detail'].apply(lambda x: x.split(" ・ ")[2])
box_office = box_office.drop('movie_detail', axis=1)
box_office

,movie_title,movie_desc,movie_rate,movie_poster,movie_year,movie_genre,movie_nation
0,문폴,"1969년 인류의 위대한 도전, 달 착륙 성공 2022년 인류 최후의 재난, 달이 ...",평균 ★2.2 (567명),https://an2-img.amz.wtchn.net/image/v2/E8ZdVl5...,2022,판타지/액션/드라마,캐나다/미국/영국
1,뜨거운 피,"부산 변두리 작은 포구 `구암`의 절대적인 주인 `손영감`(김갑수), 그의 밑에서 ...",,https://an2-img.amz.wtchn.net/image/v2/alJx8BY...,2020,범죄/드라마,한국
2,스펜서,"1991년, 왕실 가족이 샌드링엄 별장에 모여서 보내는 크리스마스. 연휴 3일간 이...","평균 ★3.6 (1,456명)",https://an2-img.amz.wtchn.net/image/v2/_8BzRXr...,2021,전기/드라마,영국/독일/미국
3,이상한 나라의 수학자,“정답보다 중요한 건 답을 찾는 과정이야” 학문의 자유를 갈망하며 탈북한 천재 수학...,"평균 ★2.9 (1,712명)",https://an2-img.amz.wtchn.net/image/v2/hsg-TCp...,2020,드라마,한국
4,더 배트맨,영웅이 될 것인가 악당이 될 것인가 운명을 결정할 선택만이 남았다 지난 2년간 고담...,평균 ★3.5 (2만명),https://an2-img.amz.wtchn.net/image/v2/NdPk_f6...,2022,액션/범죄/드라마,미국
5,극장판 주술회전 0,어릴 적 소꿉친구인 오리모토 리카를 교통사고로 눈앞에서 잃은 옷코츠 유타. “약속해...,"평균 ★3.8 (3,946명)",https://an2-img.amz.wtchn.net/image/v2/5CSy5j_...,2021,애니메이션/액션/모험,일본
6,메리 미,선결혼 후연애! 슈퍼스타 X 슈퍼노멀 커플 탄생! 슈퍼스타 `캣 발데즈’는 화려한 ...,평균 ★2.8 (65명),https://an2-img.amz.wtchn.net/image/v2/nPAFcWi...,2022,로맨스/코미디,미국
7,도어맨,해병대 복무 중 테러집단의 공격으로 대사와 가족을 잃은 알리(루비 로즈)는 제대 후...,평균 ★2.2 (165명),https://an2-img.amz.wtchn.net/image/v2/-a_wFi_...,2020,액션/스릴러,미국
8,하늘의 푸르름을 아는 사람이여,"“후회하지 않기 위해, 다시 널 만나러 왔어” ​ 산으로 둘러 싸인 시골 마을, 그...",평균 ★3.2 (365명),https://an2-img.amz.wtchn.net/image/v2/mPP0dI5...,2019,판타지/로맨스/애니메이션,일본
9,고양이들의 아파트,"서울 동쪽 끝, 거대한 아파트 단지. 그곳은 오래도록 고양이들과 사람들이 함께 마음...",평균 ★3.3 (115명),https://an2-img.amz.wtchn.net/image/v2/c3TXrIj...,2020,다큐멘터리,한국


## movie_rate에서 평점만 추출하기

In [27]:
box_office['movie_rate'] = box_office['movie_rate'].str.findall("\d+\.\d+") # 정규표현식. 언젠가 다룰 예정.
box_office

,movie_title,movie_desc,movie_rate,movie_poster,movie_year,movie_genre,movie_nation
0,문폴,"1969년 인류의 위대한 도전, 달 착륙 성공 2022년 인류 최후의 재난, 달이 ...",[2.2],https://an2-img.amz.wtchn.net/image/v2/E8ZdVl5...,2022,판타지/액션/드라마,캐나다/미국/영국
1,뜨거운 피,"부산 변두리 작은 포구 `구암`의 절대적인 주인 `손영감`(김갑수), 그의 밑에서 ...",[],https://an2-img.amz.wtchn.net/image/v2/alJx8BY...,2020,범죄/드라마,한국
2,스펜서,"1991년, 왕실 가족이 샌드링엄 별장에 모여서 보내는 크리스마스. 연휴 3일간 이...",[3.6],https://an2-img.amz.wtchn.net/image/v2/_8BzRXr...,2021,전기/드라마,영국/독일/미국
3,이상한 나라의 수학자,“정답보다 중요한 건 답을 찾는 과정이야” 학문의 자유를 갈망하며 탈북한 천재 수학...,[2.9],https://an2-img.amz.wtchn.net/image/v2/hsg-TCp...,2020,드라마,한국
4,더 배트맨,영웅이 될 것인가 악당이 될 것인가 운명을 결정할 선택만이 남았다 지난 2년간 고담...,[3.5],https://an2-img.amz.wtchn.net/image/v2/NdPk_f6...,2022,액션/범죄/드라마,미국
5,극장판 주술회전 0,어릴 적 소꿉친구인 오리모토 리카를 교통사고로 눈앞에서 잃은 옷코츠 유타. “약속해...,[3.8],https://an2-img.amz.wtchn.net/image/v2/5CSy5j_...,2021,애니메이션/액션/모험,일본
6,메리 미,선결혼 후연애! 슈퍼스타 X 슈퍼노멀 커플 탄생! 슈퍼스타 `캣 발데즈’는 화려한 ...,[2.8],https://an2-img.amz.wtchn.net/image/v2/nPAFcWi...,2022,로맨스/코미디,미국
7,도어맨,해병대 복무 중 테러집단의 공격으로 대사와 가족을 잃은 알리(루비 로즈)는 제대 후...,[2.2],https://an2-img.amz.wtchn.net/image/v2/-a_wFi_...,2020,액션/스릴러,미국
8,하늘의 푸르름을 아는 사람이여,"“후회하지 않기 위해, 다시 널 만나러 왔어” ​ 산으로 둘러 싸인 시골 마을, 그...",[3.2],https://an2-img.amz.wtchn.net/image/v2/mPP0dI5...,2019,판타지/로맨스/애니메이션,일본
9,고양이들의 아파트,"서울 동쪽 끝, 거대한 아파트 단지. 그곳은 오래도록 고양이들과 사람들이 함께 마음...",[3.3],https://an2-img.amz.wtchn.net/image/v2/c3TXrIj...,2020,다큐멘터리,한국


In [28]:
box_office['movie_rate'] = box_office['movie_rate'].apply(lambda x: x[0] if x != [] else None) # 삼항 연산자 설명
box_office

,movie_title,movie_desc,movie_rate,movie_poster,movie_year,movie_genre,movie_nation
0,문폴,"1969년 인류의 위대한 도전, 달 착륙 성공 2022년 인류 최후의 재난, 달이 ...",2.2,https://an2-img.amz.wtchn.net/image/v2/E8ZdVl5...,2022,판타지/액션/드라마,캐나다/미국/영국
1,뜨거운 피,"부산 변두리 작은 포구 `구암`의 절대적인 주인 `손영감`(김갑수), 그의 밑에서 ...",None,https://an2-img.amz.wtchn.net/image/v2/alJx8BY...,2020,범죄/드라마,한국
2,스펜서,"1991년, 왕실 가족이 샌드링엄 별장에 모여서 보내는 크리스마스. 연휴 3일간 이...",3.6,https://an2-img.amz.wtchn.net/image/v2/_8BzRXr...,2021,전기/드라마,영국/독일/미국
3,이상한 나라의 수학자,“정답보다 중요한 건 답을 찾는 과정이야” 학문의 자유를 갈망하며 탈북한 천재 수학...,2.9,https://an2-img.amz.wtchn.net/image/v2/hsg-TCp...,2020,드라마,한국
4,더 배트맨,영웅이 될 것인가 악당이 될 것인가 운명을 결정할 선택만이 남았다 지난 2년간 고담...,3.5,https://an2-img.amz.wtchn.net/image/v2/NdPk_f6...,2022,액션/범죄/드라마,미국
5,극장판 주술회전 0,어릴 적 소꿉친구인 오리모토 리카를 교통사고로 눈앞에서 잃은 옷코츠 유타. “약속해...,3.8,https://an2-img.amz.wtchn.net/image/v2/5CSy5j_...,2021,애니메이션/액션/모험,일본
6,메리 미,선결혼 후연애! 슈퍼스타 X 슈퍼노멀 커플 탄생! 슈퍼스타 `캣 발데즈’는 화려한 ...,2.8,https://an2-img.amz.wtchn.net/image/v2/nPAFcWi...,2022,로맨스/코미디,미국
7,도어맨,해병대 복무 중 테러집단의 공격으로 대사와 가족을 잃은 알리(루비 로즈)는 제대 후...,2.2,https://an2-img.amz.wtchn.net/image/v2/-a_wFi_...,2020,액션/스릴러,미국
8,하늘의 푸르름을 아는 사람이여,"“후회하지 않기 위해, 다시 널 만나러 왔어” ​ 산으로 둘러 싸인 시골 마을, 그...",3.2,https://an2-img.amz.wtchn.net/image/v2/mPP0dI5...,2019,판타지/로맨스/애니메이션,일본
9,고양이들의 아파트,"서울 동쪽 끝, 거대한 아파트 단지. 그곳은 오래도록 고양이들과 사람들이 함께 마음...",3.3,https://an2-img.amz.wtchn.net/image/v2/c3TXrIj...,2020,다큐멘터리,한국


## 파일로 저장하기

In [ ]:
box_office.to_csv("box_office.csv", index=False)

In [ ]:
box_office.to_excel("box_office.xlsx", index=False)

## 파일 불러오기

In [31]:
bodf = pd.read_csv('box_office.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'box_office.csv'

## 폰트 설정하기

In [29]:
# 운영체제별 한글 폰트 설정
import platform
import matplotlib.pyplot as plt
if platform.system() == 'Darwin': # Mac 환경 폰트 설정
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows': # Windows 환경 폰트 설정
    plt.rc('font', family='Malgun Gothic')

## 영화별 평점 Bar plot 그리기

In [30]:
bodf.plot.bar(x='movie_title',y='movie_rate')

NameError: name 'bodf' is not defined

## 인스타그램은 어떻게 크롤링할까?

### Selenium chrome driver 설치하기 [Chrome driver 다운로드](https://sites.google.com/chromium.org/driver/)
#### Windows : C:\Windows 에 넣기
#### Mac : /usr/local/bin 에 넣기

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import os
chrome_options = webdriver.ChromeOptions()
# 브라우저에 임의로 User-agent 옵션을 넣어 Python 코드로 접속함을 숨김
chrome_options.add_argument('--user-agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36"')
driver = webdriver.Chrome(chrome_options=chrome_options)
URL = "https://www.instagram.com/"

C:\Users\82104\AppData\Local\Temp/ipykernel_5764/4292658407.py:7: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options)


In [11]:
driver.get(URL)

In [9]:
import login_info

In [10]:
ID = login_info.ID
PW = login_info.PW
driver.find_element(By.NAME, "username").send_keys(ID)
driver.find_element(By.NAME, "password").send_keys(PW)

In [11]:
driver.find_element(By.CLASS_NAME, "L3NKy").click()

In [12]:
driver.find_element(By.CLASS_NAME, "L3NKy").click()

In [13]:
driver.find_element(By.CLASS_NAME, "aOOlW").click()

In [14]:
driver.get('https://www.instagram.com/'+login_info.ID)

In [ ]:
for idx, img in enumerate([i.get_attribute('src') for i in driver.find_elements(By.XPATH,'//div[@class="KL4Bh"]/img')]):
    img_data = requests.get(img).content
    os.makedirs("./profile_images", exist_ok=True) 
    with open('profile_images/'+str(idx)+'.jpg', 'wb') as handler:
        handler.write(img_data)


In [ ]:
driver.close()